In [34]:
import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List
import uuid

In [35]:
#create source file from .json currently in 1nf
df = pd.read_json('./data/tickets.json', lines=True)
#create a csv for d_airlines by extrating the dict values into a new table
from pandas.io.json import json_normalize

d_airline_df=pd.concat([pd.json_normalize(df['airline'][key]) for key in df.index]).reset_index(drop=True)
d_airline_df = d_airline_df.rename(columns={'name': 'airline_name', 'icao': 'airline_icao', 'iata': 'airline_iata', 'country': 'airline_country'})
d_airline_df.to_csv('./data/d_airlines.csv', mode= 'w')
#create a csv for d_origin_airport by extrating the dict values into a new table
d_origin_airport_df=pd.json_normalize(df['origin'])
d_origin_airport_df = d_origin_airport_df.rename(columns={'name': 'origin_airport_name', 'icao': 'origin_icao', 'iata': 'origin_airport_iata', 'country': 'origin_airport_country','city':'origin_airport_city','latitude':'origin_lat','longitude': 'origin_long', 'altitude': 'origin_altitude', 'tz_timezone':'origin_timezone'})
d_origin_airport_df.to_csv('./data/d_origin_airports.csv', mode= 'w')
#create a csv for d_destination_airports
d_destination_df=d_destination_df=pd.json_normalize(df['destination'])
d_destination_df = d_destination_df.rename(columns={'name': 'dest_airport_name', 'icao': 'dest_icao', 'iata': 'dest_airport_iata', 'country': 'dest_airport_country','city':'dest_airport_city','latitude':'dest_lat','longitude': 'dest_long', 'altitude': 'dest_altitude', 'tz_timezone':'dest_timezone'})
#create a csv for d_passenger by extrating the dict values into a new table
d_passenger_df=pd.concat([pd.json_normalize(df['passenger'][key]) for key in df.index]).reset_index(drop=True)
#add start_date and end_date columns
d_passenger_df["start_date"] = " "
d_passenger_df["end_date"] = " "
d_origin_airport_df = d_origin_airport_df.rename(columns={'name': 'origin_airport_name'})
d_passenger_df.to_csv('./data/SCD2_d_passengers.csv', mode= 'w')
#create f_tickets table
f_tickets_df = df[['eticket_num', 'confirmation','ticket_date','price','seat','status']]
#merge datasets to make a source csv
source_df= pd.merge(d_airline_df, d_origin_airport_df, left_index=True, right_index=True)
source_df= pd.merge(source_df, d_passenger_df, left_index=True, right_index=True)
source_df= pd.merge(source_df,d_destination_df, left_index=True, right_index=True)
source_df= pd.merge(source_df,f_tickets_df, left_index=True, right_index=True)
source_df["org_iata_dest_iata"] = source_df['origin_airport_iata'].astype(str) +"-"+ source_df["dest_airport_iata"] #create the primary key using both iata codes
source_df.to_csv('./data/airport_source.csv')
print(source_df.info())
source_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4096 entries, 0 to 4095
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   airline_name            4096 non-null   object 
 1   airline_iata            4096 non-null   object 
 2   airline_icao            4096 non-null   object 
 3   callsign                4096 non-null   object 
 4   airline_country         4096 non-null   object 
 5   origin_airport_name     4041 non-null   object 
 6   origin_airport_city     4041 non-null   object 
 7   origin_airport_country  4041 non-null   object 
 8   origin_airport_iata     4041 non-null   object 
 9   origin_icao             4041 non-null   object 
 10  origin_lat              4041 non-null   float64
 11  origin_long             4041 non-null   float64
 12  origin_altitude         4041 non-null   float64
 13  origin_timezone         3998 non-null   object 
 14  first_name              4096 non-null   

,airline_name,airline_iata,airline_icao,callsign,airline_country,origin_airport_name,origin_airport_city,origin_airport_country,origin_airport_iata,origin_icao,...,dest_long,dest_altitude,dest_timezone,eticket_num,confirmation,ticket_date,price,seat,status,org_iata_dest_iata
0,China Eastern Airlines,MU,CES,CHINA EASTERN,China,Montreal / Pierre Elliott Trudeau Internationa...,Montreal,Canada,YUL,CYUL,...,-87.75,620.0,America/Chicago,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,YUL-MDW
1,Hawaiian Airlines,HA,HAL,HAWAIIAN,United States,Longdongbao Airport,Guiyang,China,KWE,ZUGY,...,121.02,75.0,Asia/Manila,482-850738-6048,IL5GUI,2022-03-23,765.18,29B,active,KWE-MNL
2,Wizz Air,W6,WZZ,WIZZ AIR,Hungary,Licenciado Gustavo Díaz Ordaz International Ai...,Puerto Vallarta,Mexico,PVR,MMPR,...,1.37,24.0,Europe/Madrid,275-207321-8092,CYEFBC,2022-03-21,753.89,26I,active,PVR-IBZ
3,AirAsia,AK,AXM,ASIAN EXPRESS,Malaysia,El Tepual Airport,Puerto Montt,Chile,PMC,SCTE,...,18.47,489.0,Europe/Warsaw,246-793315-3102,ZNGPC2,2022-03-22,793.89,15A,active,PMC-GDN
4,Xiamen Airlines,MF,CXA,XIAMEN AIR,China,Baltimore/Washington International Thurgood Ma...,Baltimore,United States,BWI,KBWI,...,-0.19,202.0,Europe/London,091-128904-1226,MGSBD9,2022-03-24,820.25,17F,active,BWI-LGW


In [36]:
import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List


# **** SETUP ****

# change to match your filesystem
DATA_DIR = "./data/"
DEFAULT_SOURCE_FILE = os.path.join(DATA_DIR, "airport_source.csv")
# change to match your gcloud project 
PROJECT_NAME = "deb-01-372116"
DATASET_NAME = "airline_ticket_processor"


# **** TABLE SCHEMAS ****

TABLE_METADATA = {
    'd_airlines': {
        'table_name': 'd_airlines',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('airline_iata', 'string', mode='REQUIRED'),
            bigquery.SchemaField('airline_name', 'string', mode='NULLABLE'),
            bigquery.SchemaField('airline_icao', 'string', mode='NULLABLE'),
            bigquery.SchemaField('callsign', 'string', mode='NULLABLE'),
            bigquery.SchemaField('airline_country', 'string', mode='NULLABLE'),
        ],
    },
    'd_airports': {
        'table_name': 'd_airports',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('org_iata_dest_iata', 'string', mode='REQUIRED'),
            bigquery.SchemaField('origin_airport_iata', 'string', mode='NULLABLE'),
            bigquery.SchemaField('dest_airport_iata', 'string', mode='NULLABLE'),
            bigquery.SchemaField('origin_airport_name', 'string', mode='NULLABLE'),
            bigquery.SchemaField('origin_airport_city', 'string', mode='NULLABLE'),
            bigquery.SchemaField('origin_airport_country', 'string', mode='NULLABLE'),
            bigquery.SchemaField('origin_icao', 'string', mode='NULLABLE'),
            bigquery.SchemaField('origin_lat', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('origin_long', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('origin_altitude', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('origin_timezone', 'string', mode='NULLABLE'),
            bigquery.SchemaField('dest_airport_name', 'string', mode='NULLABLE'),
            bigquery.SchemaField('dest_airport_city', 'string', mode='NULLABLE'),
            bigquery.SchemaField('dest_airport_country', 'string', mode='NULLABLE'),
            bigquery.SchemaField('dest_icao', 'string', mode='NULLABLE'),
            bigquery.SchemaField('dest_lat', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('dest_long', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('dest_altitude', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('dest_timezone', 'string', mode='NULLABLE'),
        ],
    },
    'SCD2_d_passengers': {
        'table_name': 'SCD2_d_passengers',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('UUID_SK', 'string', mode='REQUIRED'),
            bigquery.SchemaField('first_name', 'string', mode='NULLABLE'),
            bigquery.SchemaField('last_name', 'string', mode='NULLABLE'),
            bigquery.SchemaField('gender', 'string', mode='NULLABLE'),
            bigquery.SchemaField('birth_date', 'string', mode='NULLABLE'),
            bigquery.SchemaField('email', 'string', mode='REQUIRED'),
            bigquery.SchemaField('city', 'string', mode='NULLABLE'),
            bigquery.SchemaField('state', 'string', mode='NULLABLE'),
            bigquery.SchemaField('zip', 'INTEGER', mode='NULLABLE'),
            bigquery.SchemaField('start_date', 'string', mode='NULLABLE'),
            bigquery.SchemaField('end_date', 'string', mode='NULLABLE'),
        ],
    },
}


# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level


# **** BIGQUERY CLIENT ****
logger.debug(f"Creating bigquery client")
client = bigquery.Client()

logger.info(f"Setup Completed")

[DEBUG][2023-01-06 15:55:07,432][4055345799:0091] : Creating bigquery client
[INFO ][2023-01-06 15:55:07,436][4055345799:0094] : Setup Completed


In [37]:
# create dataset if needed
dataset_id = f"{PROJECT_NAME}.{DATASET_NAME}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)

logger.info(f"Created airline_ticket_processor: {dataset.full_dataset_id}")

[INFO ][2023-01-06 15:55:09,054][2378520198:0007] : Created airline_ticket_processor: deb-01-372116:airline_ticket_processor


In [38]:
#load airlines data
# receipts data file name
filename = DEFAULT_SOURCE_FILE
logger.debug(f"attempting to process: {filename}")

# *** always perform checks first ***
# check if the file exists
assert os.path.exists(filename), f"Data file does not exists: '{filename}'"
# check if the file contains any data. Header row alone is 78 bytes. size > 78
assert os.path.getsize(filename) > 78, f"Data file size incorrect; does not seem to contain data: '{filename}'"

# load into dataframe
df = pd.read_csv(filename, header=0)
logger.info(f"loaded {len(df.index)} rows from: {filename}")

# assign & remember receipts dataframe
source_df = df
display(source_df.head(n=10))

[DEBUG][2023-01-06 15:55:09,113][2025020176:0004] : attempting to process: ./data/airport_source.csv
[INFO ][2023-01-06 15:55:09,196][2025020176:0014] : loaded 4096 rows from: ./data/airport_source.csv


,Unnamed: 0,airline_name,airline_iata,airline_icao,callsign,airline_country,origin_airport_name,origin_airport_city,origin_airport_country,origin_airport_iata,...,dest_long,dest_altitude,dest_timezone,eticket_num,confirmation,ticket_date,price,seat,status,org_iata_dest_iata
0,0,China Eastern Airlines,MU,CES,CHINA EASTERN,China,Montreal / Pierre Elliott Trudeau Internationa...,Montreal,Canada,YUL,...,-87.75,620.0,America/Chicago,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,YUL-MDW
1,1,Hawaiian Airlines,HA,HAL,HAWAIIAN,United States,Longdongbao Airport,Guiyang,China,KWE,...,121.02,75.0,Asia/Manila,482-850738-6048,IL5GUI,2022-03-23,765.18,29B,active,KWE-MNL
2,2,Wizz Air,W6,WZZ,WIZZ AIR,Hungary,Licenciado Gustavo Díaz Ordaz International Ai...,Puerto Vallarta,Mexico,PVR,...,1.37,24.0,Europe/Madrid,275-207321-8092,CYEFBC,2022-03-21,753.89,26I,active,PVR-IBZ
3,3,AirAsia,AK,AXM,ASIAN EXPRESS,Malaysia,El Tepual Airport,Puerto Montt,Chile,PMC,...,18.47,489.0,Europe/Warsaw,246-793315-3102,ZNGPC2,2022-03-22,793.89,15A,active,PMC-GDN
4,4,Xiamen Airlines,MF,CXA,XIAMEN AIR,China,Baltimore/Washington International Thurgood Ma...,Baltimore,United States,BWI,...,-0.19,202.0,Europe/London,091-128904-1226,MGSBD9,2022-03-24,820.25,17F,active,BWI-LGW
5,5,Air New Zealand,NZ,ANZ,NEW ZEALAND,New Zealand,Platov International Airport,Rostov,Russia,ROV,...,-84.67,896.0,America/New_York,115-196069-8963,XFYQC0,2022-03-23,892.69,18C,active,ROV-CVG
6,6,Jeju Air,7C,JJA,JEJU AIR,Republic of Korea,Winnipeg / James Armstrong Richardson Internat...,Winnipeg,Canada,YWG,...,127.65,12.0,Asia/Tokyo,396-673460-1326,N5UOOZ,2022-03-23,889.53,3C,active,YWG-OKA
7,7,American Airlines,AA,AAL,AMERICAN,United States,Ontario International Airport,Ontario,United States,ONT,...,96.13,109.0,Asia/Rangoon,380-894599-8109,PAA19Y,2022-03-22,706.78,7D,active,ONT-RGN
8,8,Juneyao Airlines,HO,DKH,JUNEYAO AIRLINES,China,Luis Munoz Marin International Airport,San Juan,Puerto Rico,SJU,...,60.80,764.0,Asia/Yekaterinburg,614-960971-2686,EF4BHJ,2022-03-23,486.40,24J,active,SJU-SVX
9,9,Royal Air Maroc,AT,RAM,ROYALAIR MAROC,Morocco,Edmonton International Airport,Edmonton,Canada,YEG,...,112.63,2575.0,Asia/Shanghai,481-321233-0702,FVM9EE,2022-03-23,855.93,16A,active,YEG-TYN


In [39]:
# prepare airline df to be loaded
df = source_df

logger.debug(f"getting uniques products...")

# set of unique columns to return
cols = ['airline_iata', 'airline_name', 'airline_icao', 'callsign', 'airline_country']
# group by unique columns and only select them
df = df.groupby(cols).all()
df = df.reset_index().loc[:, cols]

logger.info(f"products dim - found {len(df.index)} rows")
display(df)

[DEBUG][2023-01-06 15:55:09,286][2107265125:0004] : getting uniques products...
[INFO ][2023-01-06 15:55:09,404][2107265125:0012] : products dim - found 48 rows


,airline_iata,airline_name,airline_icao,callsign,airline_country
0,3U,Sichuan Airlines,CSC,SI CHUAN,China
1,7C,Jeju Air,JJA,JEJU AIR,Republic of Korea
2,9K,Cape Air,KAP,CAIR,United States
3,9S,Spring Airlines,CQH,AIR SPRING,China
4,AA,American Airlines,AAL,AMERICAN,United States
5,AC,Air Canada,ACA,AIR CANADA,Canada
6,AF,Air France,AFR,AIRFRANS,France
7,AK,AirAsia,AXM,ASIAN EXPRESS,Malaysia
8,AS,Alaska Airlines,ASA,Inc.,ALASKA
9,AT,Royal Air Maroc,RAM,ROYALAIR MAROC,Morocco


In [40]:
#define the load_table function
def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """
    # *** run some checks ***
    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish
    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

In [41]:
##----------------load d_airlines---------------------
# get table name and schema from our TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['d_airlines']['table_name']}"
schema = schema=TABLE_METADATA['d_airlines']['schema']
# load dataframe
load_table(df, client, table_name, schema)

logger.info(f"loaded products dim")

[INFO ][2023-01-06 15:55:09,582][1547112885:0030] : loading table: 'deb-01-372116.airline_ticket_processor.d_airlines'
[INFO ][2023-01-06 15:55:14,519][1547112885:0035] : loaded 48 rows into deb-01-372116:airline_ticket_processor.d_airlines
[INFO ][2023-01-06 15:55:14,520][3048314498:0008] : loaded products dim


In [42]:
#----------------load d_airports---------------------
df = source_df

logger.debug(f"getting uniques airports...")

# set of unique columns to return
cols = ['org_iata_dest_iata','origin_airport_iata', 'dest_airport_iata', 'origin_airport_name', 'origin_airport_city', 'origin_airport_country', 'origin_icao', 'origin_lat','origin_long','origin_altitude','origin_timezone','dest_airport_name', 'dest_airport_city', 'dest_airport_country', 'dest_icao', 'dest_lat','dest_long','dest_altitude','dest_timezone']
# group by unique columns and only select them
df = df.groupby(cols).all()
df = df.reset_index().loc[:, cols]
logger.info(f"generated iata codes")

logger.info(f"airports dim - found {len(df.index)} rows")
display(df)

#load table into bigquery
# get table name and schema from our TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['d_airports']['table_name']}"
schema = schema=TABLE_METADATA['d_airports']['schema']
# load dataframe
load_table(df, client, table_name, schema)

logger.info(f"loaded airports dim")

[DEBUG][2023-01-06 15:55:14,623][2128840201:0004] : getting uniques airports...
[INFO ][2023-01-06 15:55:14,721][2128840201:0012] : generated iata codes
[INFO ][2023-01-06 15:55:14,722][2128840201:0014] : airports dim - found 3817 rows


,org_iata_dest_iata,origin_airport_iata,dest_airport_iata,origin_airport_name,origin_airport_city,origin_airport_country,origin_icao,origin_lat,origin_long,origin_altitude,origin_timezone,dest_airport_name,dest_airport_city,dest_airport_country,dest_icao,dest_lat,dest_long,dest_altitude,dest_timezone
0,ABQ-BOM,ABQ,BOM,Albuquerque International Sunport,Albuquerque,United States,KABQ,35.04,-106.61,5355.0,America/Denver,Chhatrapati Shivaji International Airport,Mumbai,India,VABB,19.09,72.87,39.0,Asia/Calcutta
1,ABQ-HKG,ABQ,HKG,Albuquerque International Sunport,Albuquerque,United States,KABQ,35.04,-106.61,5355.0,America/Denver,Hong Kong International Airport,Hong Kong,Hong Kong,VHHH,22.31,113.92,28.0,Asia/Hong_Kong
2,ABQ-JUL,ABQ,JUL,Albuquerque International Sunport,Albuquerque,United States,KABQ,35.04,-106.61,5355.0,America/Denver,Inca Manco Capac International Airport,Juliaca,Peru,SPJL,-15.47,-70.16,12552.0,America/Lima
3,ABQ-KUL,ABQ,KUL,Albuquerque International Sunport,Albuquerque,United States,KABQ,35.04,-106.61,5355.0,America/Denver,Kuala Lumpur International Airport,Kuala Lumpur,Malaysia,WMKK,2.75,101.71,69.0,Asia/Kuala_Lumpur
4,ABQ-MAD,ABQ,MAD,Albuquerque International Sunport,Albuquerque,United States,KABQ,35.04,-106.61,5355.0,America/Denver,Adolfo Suárez Madrid–Barajas Airport,Madrid,Spain,LEMD,40.47,-3.56,1998.0,Europe/Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3812,ZUH-PMC,ZUH,PMC,Zhuhai Jinwan Airport,Zhuhai,China,ZGSD,22.01,113.38,23.0,Asia/Shanghai,El Tepual Airport,Puerto Montt,Chile,SCTE,-41.44,-73.09,294.0,America/Santiago
3813,ZUH-RDU,ZUH,RDU,Zhuhai Jinwan Airport,Zhuhai,China,ZGSD,22.01,113.38,23.0,Asia/Shanghai,Raleigh Durham International Airport,Raleigh-durham,United States,KRDU,35.88,-78.79,435.0,America/New_York
3814,ZUH-RSW,ZUH,RSW,Zhuhai Jinwan Airport,Zhuhai,China,ZGSD,22.01,113.38,23.0,Asia/Shanghai,Southwest Florida International Airport,Fort Myers,United States,KRSW,26.54,-81.76,30.0,America/New_York
3815,ZUH-XIY,ZUH,XIY,Zhuhai Jinwan Airport,Zhuhai,China,ZGSD,22.01,113.38,23.0,Asia/Shanghai,Xi'an Xianyang International Airport,Xi'an,China,ZLXY,34.45,108.75,1572.0,Asia/Shanghai


[INFO ][2023-01-06 15:55:14,744][1547112885:0030] : loading table: 'deb-01-372116.airline_ticket_processor.d_airports'
[INFO ][2023-01-06 15:55:20,605][1547112885:0035] : loaded 3817 rows into deb-01-372116:airline_ticket_processor.d_airports
[INFO ][2023-01-06 15:55:20,607][2128840201:0024] : loaded airports dim


In [43]:
#----------------load SCD2_d_passengers---------------------
df = source_df

logger.debug(f"getting unique passengers...")

#create UUIDS
for email in df['email'].unique():
    df.loc[df['email'] == email, 'UUID_SK'] = uuid.uuid4()
df = df.astype({"UUID_SK": str})
#set start-date to the ticket date
df = df.assign(start_date=f_tickets_df['ticket_date'])
df = df.assign(end_date='None')
# set of unique columns to return
cols = ['UUID_SK','email','first_name','last_name', 'gender', 'birth_date', 'street', 'city', 'state','zip','start_date','end_date']
# group by unique columns and only select them
df = df.groupby(cols).all()
df = df.reset_index().loc[:, cols]

logger.info(f"passengers dim - found {len(df.index)} rows")

# #load table into bigquery
# get table name and schema from our TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['SCD2_d_passengers']['table_name']}"
schema = schema=TABLE_METADATA['SCD2_d_passengers']['schema']
# load dataframe
load_table(df, client, table_name, schema)

logger.info(f"loaded passengers dim")

display(df.head())

[DEBUG][2023-01-06 15:55:20,666][756748291:0004] : getting unique passengers...
[INFO ][2023-01-06 15:55:20,801][756748291:0019] : passengers dim - found 128 rows
[INFO ][2023-01-06 15:55:20,802][1547112885:0030] : loading table: 'deb-01-372116.airline_ticket_processor.SCD2_d_passengers'
[INFO ][2023-01-06 15:55:23,881][1547112885:0035] : loaded 128 rows into deb-01-372116:airline_ticket_processor.SCD2_d_passengers
[INFO ][2023-01-06 15:55:23,882][756748291:0028] : loaded passengers dim


,UUID_SK,email,first_name,last_name,gender,birth_date,street,city,state,zip,start_date,end_date
0,00e3548b-f105-40b4-99c8-eb388d64bc24,ronald.cook.76@hotmail.com,Ronald,Cook,M,1976-07-29,93328 Davis Island,Rodriguezside,MD,21408,2022-03-21,None
1,00e3548b-f105-40b4-99c8-eb388d64bc24,ronald.cook.76@hotmail.com,Ronald,Cook,M,1976-07-29,93328 Davis Island,Rodriguezside,MD,21408,2022-03-22,None
2,00e3548b-f105-40b4-99c8-eb388d64bc24,ronald.cook.76@hotmail.com,Ronald,Cook,M,1976-07-29,93328 Davis Island,Rodriguezside,MD,21408,2022-03-23,None
3,00e3548b-f105-40b4-99c8-eb388d64bc24,ronald.cook.76@hotmail.com,Ronald,Cook,M,1976-07-29,93328 Davis Island,Rodriguezside,MD,21408,2022-03-24,None
4,028bc9fb-d4cd-4bef-8c23-2012cf502fe1,hannah.smith.66@gmail.com,Hannah,Smith,F,1966-07-01,230 Donna Street,Lake Adrianstad,MN,56413,2022-03-21,None
